In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jan  6 11:35:20 2019

@author: RB
"""
from kafka import KafkaConsumer
from pymongo import MongoClient
from json import loads
from datetime import datetime

print('\nStart time of consumer program: ', datetime.now().strftime("%c"))

countDocsWritten1 = 0
countDocsWritten2 = 0
msgsReadCount = 0
topicName = 'TestNYTFullJuneDb10'

consumer = KafkaConsumer(
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
#     auto_offset_reset='latest',   # default is latest
     enable_auto_commit=True,
     auto_commit_interval_ms=2000,
     group_id='my-group1')
#     consumer_timeout_ms=5000)   #consumer_timeout_ms (int) – number of milliseconds to block during 
                                 #    message iteration before raising StopIteration (i.e., ending the 
                                 #    iterator). Default block forever [float(‘inf’)].
consumer.subscribe(topicName)

client1 = MongoClient('localhost:27017')
collection1 = client.TestNYTFullJuneDb10.TestNYTFullJuneColJun01to12
collection2 = client.TestNYTFullJuneDb10.TestNYTFullJuneColJun13to24

boundLowerPUDatetime1 = '2018-06-01 00:00:00'   # the pickup datetime from csv to be greater or equal to this
boundUpperPUDatetime1 = '2018-06-13 00:00:00'   # the pickup datetime from csv to be lower than this
boundLowerPUDatetime2 = '2018-06-13 00:00:00'   # the pickup datetime from csv to be greater or equal to this
boundUpperPUDatetime2 = '2018-06-25 00:00:00'   # the pickup datetime from csv to be lower than this

for message in consumer:
    msgsReadCount = msgsReadCount + 1
    NYTNewRowJsonBuiltUpAsString = "{}"
#    print(f'Full message received from Kafka::\n', message)
    msgAsString = message.value.decode("utf-8")
    if msgAsString == '':
        print('IGNORED SECOND ROW with all commans, CONTINUING to end of the for loop')
        continue
    elif msgAsString[0:8] == 'VendorID':
        print('IGNORED FIRST ROW with columns names, CONTINUING to end of the for loop')
        continue
    elif msgAsString.count(',') == 16:
        msgAsList = msgAsString.split(",")
        trip_distanceAsString = str(msgAsList[4])
        if trip_distanceAsString[0:1] == '.':        # a 0 value comes as  .00  so checking and making as 0.00
            trip_distanceAsString = '0' + trip_distanceAsString
        NYTNewRowJsonBuiltUpAsString='{' +                                      \
        '"VendorID" : ' + str(msgAsList[0]) + ', ' +                            \
        '"tpep_pickup_datetime" : ' + '"' + str(msgAsList[1]) +  '"' + ', ' +   \
        '"tpep_dropoff_datetime" : ' + '"' + str(msgAsList[2]) +  '"' + ', ' +  \
        '"passenger_count" : ' + str(msgAsList[3]) + ', ' +                     \
        '"trip_distance" : ' + trip_distanceAsString + ', ' +                   \
        '"RatecodeID" : ' + str(msgAsList[5]) + ', ' +                          \
        '"store_and_fwd_flag" : ' + '"' + str(msgAsList[6]) + '"' + ', ' +      \
        '"PULocationID" : ' + str(msgAsList[7]) + ', ' +                        \
        '"DOLocationID" : ' + str(msgAsList[8]) + ', ' +                        \
        '"payment_type" : ' + str(msgAsList[9]) + ', ' +                        \
        '"fare_amount" : ' + str(msgAsList[10]) + ', ' +                        \
        '"extra" : ' + str(msgAsList[11]) + ', ' +                              \
        '"mta_tax" : ' + str(msgAsList[12]) + ', ' +                            \
        '"tip_amount" : ' + str(msgAsList[13]) + ', ' +                         \
        '"tolls_amount" : ' + str(msgAsList[14]) + ', ' +                       \
        '"improvement_surcharge" : ' + str(msgAsList[15]) + ', ' +              \
        '"total_amount" : ' + str(msgAsList[16]) + '}'
        #print(f'Msg {msgsReadCount} - JSON built as=\n{NYTNewRowJsonBuiltUpAsString}')
        if ( str(msgAsList[1]) < boundUpperPUDatetime1 and str(msgAsList[1]) >= boundLowerPUDatetime1 ):
            try:
                collection1.insert_one(loads(NYTNewRowJsonBuiltUpAsString))
                #print(f"Collection 1 -- NO ERROR for {msgsReadCount} th message insert")
                countDocsWritten1 = countDocsWritten1 + 1
                if countDocsWritten1 % 50000 == 0:
                    print(f"Collection 1 - write SUCCESS - {msgsReadCount} th message as {countDocsWritten1} th insert")
                if countDocsWritten1 % 250000 == 0:
                    print(f'Msg {msgsReadCount} - JSON built as=\n{NYTNewRowJsonBuiltUpAsString}')
            except Exception as e:
                print(f'Collection 1 ERROR Insert :: {msgsReadCount} th message\nError:: ', type(e), e)
        elif ( str(msgAsList[1]) < boundUpperPUDatetime2 and str(msgAsList[1]) >= boundLowerPUDatetime2 ):
            try:
                collection1.insert_one(loads(NYTNewRowJsonBuiltUpAsString))
                #print(f"Collection 2 -- NO ERROR for {msgsReadCount} th message insert")
                countDocsWritten2 = countDocsWritten2 + 1
                if countDocsWritten2 % 50000 == 0:
                    print(f"Collection 2 - write SUCCESS - {msgsReadCount} th message as {countDocsWritten2} th insert")
                if countDocsWritten2 % 500000 == 0:
                    print(f'Msg {msgsReadCount} - JSON built as=\n{NYTNewRowJsonBuiltUpAsString}')
            except Exception as e:
                print(f'Collection 2 ERROR Insert :: {msgsReadCount} th message\nError:: ', type(e), e)
    else:
        print(f"{msgsReadCount} th message NOT having 16 commas so IGNORED MESSAGE")


print('\nExited FOR LOOP\nRead total %d messages from Kafka' %(msgsReadCount))
print('\nWritten %d documents to Mongo Collection1' %(countDocsWritten1))
print('\nWritten %d documents to Mongo Collection2' %(countDocsWritten2))
print('\nDone processing. Normal exit from program at: ', datetime.now().strftime("%c"))
#
# ways to copy a dictionary value      dict2 = dict(dict1)   OR   dict2 = dict1.copy()
#


Start time of consumer program:  Sun Jan  6 12:21:44 2019


NameError: name 'client' is not defined

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Jan  6 11:35:20 2019

@author: RB
"""
from kafka import KafkaConsumer
from pymongo import MongoClient
from json import loads
from datetime import datetime

print('\nStart time of consumer program: ', datetime.now().strftime("%c"))

countDocsWritten1 = 0
countDocsWritten2 = 0
msgsReadCount = 0
topicName = 'TestNYTFullJuneDb10'

consumer = KafkaConsumer(
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
#     auto_offset_reset='latest',   # default is latest
     enable_auto_commit=True,
     auto_commit_interval_ms=2000,
     group_id='my-group1')
#     consumer_timeout_ms=5000)   #consumer_timeout_ms (int) – number of milliseconds to block during 
                                 #    message iteration before raising StopIteration (i.e., ending the 
                                 #    iterator). Default block forever [float(‘inf’)].
consumer.subscribe(topicName)

client1 = MongoClient('localhost:27017')
client2 = MongoClient('localhost:27017')
collection1 = client1.TestNYTFullJuneDb10.TestNYTFullJuneColJun01to12
collection2 = client2.TestNYTFullJuneDb10.TestNYTFullJuneColJun13to24

boundLowerPUDatetime1 = '2018-06-01 00:00:00'   # the pickup datetime from csv to be greater or equal to this
boundUpperPUDatetime1 = '2018-06-13 00:00:00'   # the pickup datetime from csv to be lower than this
boundLowerPUDatetime2 = '2018-06-13 00:00:00'   # the pickup datetime from csv to be greater or equal to this
boundUpperPUDatetime2 = '2018-06-25 00:00:00'   # the pickup datetime from csv to be lower than this

for message in consumer:
    msgsReadCount = msgsReadCount + 1
    NYTNewRowJsonBuiltUpAsString = "{}"
#    print(f'Full message received from Kafka::\n', message)
    msgAsString = message.value.decode("utf-8")
    if msgAsString == '':
        print('IGNORED SECOND ROW with all commans, CONTINUING to end of the for loop')
        continue
    elif msgAsString[0:8] == 'VendorID':
        print('IGNORED FIRST ROW with columns names, CONTINUING to end of the for loop')
        continue
    elif msgAsString.count(',') == 16:
        msgAsList = msgAsString.split(",")
        trip_distanceAsString = str(msgAsList[4])
        if trip_distanceAsString[0:1] == '.':        # a 0 value comes as  .00  so checking and making as 0.00
            trip_distanceAsString = '0' + trip_distanceAsString
        NYTNewRowJsonBuiltUpAsString='{' +                                      \
        '"VendorID" : ' + str(msgAsList[0]) + ', ' +                            \
        '"tpep_pickup_datetime" : ' + '"' + str(msgAsList[1]) +  '"' + ', ' +   \
        '"tpep_dropoff_datetime" : ' + '"' + str(msgAsList[2]) +  '"' + ', ' +  \
        '"passenger_count" : ' + str(msgAsList[3]) + ', ' +                     \
        '"trip_distance" : ' + trip_distanceAsString + ', ' +                   \
        '"RatecodeID" : ' + str(msgAsList[5]) + ', ' +                          \
        '"store_and_fwd_flag" : ' + '"' + str(msgAsList[6]) + '"' + ', ' +      \
        '"PULocationID" : ' + str(msgAsList[7]) + ', ' +                        \
        '"DOLocationID" : ' + str(msgAsList[8]) + ', ' +                        \
        '"payment_type" : ' + str(msgAsList[9]) + ', ' +                        \
        '"fare_amount" : ' + str(msgAsList[10]) + ', ' +                        \
        '"extra" : ' + str(msgAsList[11]) + ', ' +                              \
        '"mta_tax" : ' + str(msgAsList[12]) + ', ' +                            \
        '"tip_amount" : ' + str(msgAsList[13]) + ', ' +                         \
        '"tolls_amount" : ' + str(msgAsList[14]) + ', ' +                       \
        '"improvement_surcharge" : ' + str(msgAsList[15]) + ', ' +              \
        '"total_amount" : ' + str(msgAsList[16]) + '}'
        #print(f'Msg {msgsReadCount} - JSON built as=\n{NYTNewRowJsonBuiltUpAsString}')
        if ( str(msgAsList[1]) < boundUpperPUDatetime1 and str(msgAsList[1]) >= boundLowerPUDatetime1 ):
            try:
                collection1.insert_one(loads(NYTNewRowJsonBuiltUpAsString))
                #print(f"Collection 1 -- NO ERROR for {msgsReadCount} th message insert")
                countDocsWritten1 = countDocsWritten1 + 1
                if countDocsWritten1 % 50000 == 0:
                    print(f"Collection 1 - write SUCCESS - {msgsReadCount} th message as {countDocsWritten1} th insert")
                if countDocsWritten1 % 250000 == 0:
                    print(f'Msg {msgsReadCount} - JSON built as=\n{NYTNewRowJsonBuiltUpAsString}')
            except Exception as e:
                print(f'Collection 1 ERROR Insert :: {msgsReadCount} th message\nError:: ', type(e), e)
        elif ( str(msgAsList[1]) < boundUpperPUDatetime2 and str(msgAsList[1]) >= boundLowerPUDatetime2 ):
            try:
                collection2.insert_one(loads(NYTNewRowJsonBuiltUpAsString))
                #print(f"Collection 2 -- NO ERROR for {msgsReadCount} th message insert")
                countDocsWritten2 = countDocsWritten2 + 1
                if countDocsWritten2 % 50000 == 0:
                    print(f"Collection 2 - write SUCCESS - {msgsReadCount} th message as {countDocsWritten2} th insert")
                if countDocsWritten2 % 500000 == 0:
                    print(f'Msg {msgsReadCount} - JSON built as=\n{NYTNewRowJsonBuiltUpAsString}')
            except Exception as e:
                print(f'Collection 2 ERROR Insert :: {msgsReadCount} th message\nError:: ', type(e), e)
    else:
        print(f"{msgsReadCount} th message NOT having 16 commas so IGNORED MESSAGE")


print('\nExited FOR LOOP\nRead total %d messages from Kafka' %(msgsReadCount))
print('\nWritten %d documents to Mongo Collection1' %(countDocsWritten1))
print('\nWritten %d documents to Mongo Collection2' %(countDocsWritten2))
print('\nDone processing. Normal exit from program at: ', datetime.now().strftime("%c"))
#
# ways to copy a dictionary value      dict2 = dict(dict1)   OR   dict2 = dict1.copy()
#


Start time of consumer program:  Sun Jan  6 12:28:23 2019
Collection 1 - write SUCCESS - 50000 th message as 50000 th insert
Collection 1 - write SUCCESS - 100000 th message as 100000 th insert
Collection 1 - write SUCCESS - 150000 th message as 150000 th insert
Collection 1 - write SUCCESS - 200000 th message as 200000 th insert
Collection 1 - write SUCCESS - 250000 th message as 250000 th insert
Msg 250000 - JSON built as=
{"VendorID" : 1, "tpep_pickup_datetime" : "2018-06-01 19:10:00", "tpep_dropoff_datetime" : "2018-06-01 19:16:31", "passenger_count" : 1, "trip_distance" : 1.20, "RatecodeID" : 1, "store_and_fwd_flag" : "N", "PULocationID" : 186, "DOLocationID" : 249, "payment_type" : 1, "fare_amount" : 6.5, "extra" : 1, "mta_tax" : 0.5, "tip_amount" : 1.65, "tolls_amount" : 0, "improvement_surcharge" : 0.3, "total_amount" : 9.95}
Collection 1 - write SUCCESS - 300000 th message as 300000 th insert
Collection 1 - write SUCCESS - 350000 th message as 350000 th insert
Collection 1 - 

Collection 1 - write SUCCESS - 2700001 th message as 2700000 th insert
Collection 1 - write SUCCESS - 2750001 th message as 2750000 th insert
Msg 2750001 - JSON built as=
{"VendorID" : 2, "tpep_pickup_datetime" : "2018-06-10 07:43:07", "tpep_dropoff_datetime" : "2018-06-10 08:06:37", "passenger_count" : 2, "trip_distance" : 9.96, "RatecodeID" : 1, "store_and_fwd_flag" : "N", "PULocationID" : 237, "DOLocationID" : 138, "payment_type" : 1, "fare_amount" : 30.5, "extra" : 0, "mta_tax" : 0.5, "tip_amount" : 7.41, "tolls_amount" : 5.76, "improvement_surcharge" : 0.3, "total_amount" : 44.47}
Collection 1 - write SUCCESS - 2800001 th message as 2800000 th insert
Collection 1 - write SUCCESS - 2850001 th message as 2850000 th insert
Collection 1 - write SUCCESS - 2900001 th message as 2900000 th insert
Collection 1 - write SUCCESS - 2950001 th message as 2950000 th insert
Collection 1 - write SUCCESS - 3000001 th message as 3000000 th insert
Msg 3000001 - JSON built as=
{"VendorID" : 2, "tpep_

Collection 2 - write SUCCESS - 6039527 th message as 2500000 th insert
Msg 6039527 - JSON built as=
{"VendorID" : 2, "tpep_pickup_datetime" : "2018-06-21 12:53:31", "tpep_dropoff_datetime" : "2018-06-21 13:24:51", "passenger_count" : 1, "trip_distance" : 7.98, "RatecodeID" : 1, "store_and_fwd_flag" : "N", "PULocationID" : 138, "DOLocationID" : 236, "payment_type" : 1, "fare_amount" : 27.5, "extra" : 0, "mta_tax" : 0.5, "tip_amount" : 8.52, "tolls_amount" : 5.76, "improvement_surcharge" : 0.3, "total_amount" : 42.58}
Collection 2 - write SUCCESS - 6089527 th message as 2550000 th insert
Collection 2 - write SUCCESS - 6139527 th message as 2600000 th insert
Collection 2 - write SUCCESS - 6189527 th message as 2650000 th insert
Collection 2 - write SUCCESS - 6239527 th message as 2700000 th insert
Collection 2 - write SUCCESS - 6289527 th message as 2750000 th insert
Collection 2 - write SUCCESS - 6339527 th message as 2800000 th insert
Collection 2 - write SUCCESS - 6389527 th message as

In [ ]:
#producer was running and started the consumer (single instance above). Run time is XX to YY.
#took XX mins to finish.
#as producer faster than consumer, don't think need to run again with producer stopped  for single instance consumer
# runtime check.

In [ ]:
#now producer has finished run but going to run THREE consumers together. Run times are:
# Consumer 1:::  XX  to  YY         Runtime: xx mins
# Consumer 2:::  XX  to  YY         Runtime: xx mins
# Consumer 3:::  XX  to  YY         Runtime: xx mins

In [ ]:
# CONSUMER 1    CONSUMER 1
# -*- coding: utf-8 -*-
"""
Created on Sun Jan  6 11:35:20 2019

@author: RB
"""
from kafka import KafkaConsumer
from pymongo import MongoClient
from json import loads
from datetime import datetime

print('\nStart time of consumer program: ', datetime.now().strftime("%c"))

countDocsWritten1 = 0
countDocsWritten2 = 0
msgsReadCount = 0
topicName = 'TestNYTFullJuneDb10

consumer = KafkaConsumer(
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
#     auto_offset_reset='latest',   # default is latest
     enable_auto_commit=True,
     auto_commit_interval_ms=2000,
     group_id='my-group1')
#     consumer_timeout_ms=5000)   #consumer_timeout_ms (int) – number of milliseconds to block during 
                                 #    message iteration before raising StopIteration (i.e., ending the 
                                 #    iterator). Default block forever [float(‘inf’)].
consumer.subscribe(topicName)

client1 = MongoClient('localhost:27017')
client2 = MongoClient('localhost:27017')
collection1 = client1.TestNYTFullJuneDb11.TestNYTFullJuneColJun01to12
collection2 = client2.TestNYTFullJuneDb11.TestNYTFullJuneColJun13to24

boundLowerPUDatetime1 = '2018-06-01 00:00:00'   # the pickup datetime from csv to be greater or equal to this
boundUpperPUDatetime1 = '2018-06-13 00:00:00'   # the pickup datetime from csv to be lower than this
boundLowerPUDatetime2 = '2018-06-13 00:00:00'   # the pickup datetime from csv to be greater or equal to this
boundUpperPUDatetime2 = '2018-06-25 00:00:00'   # the pickup datetime from csv to be lower than this

for message in consumer:
    msgsReadCount = msgsReadCount + 1
    NYTNewRowJsonBuiltUpAsString = "{}"
#    print(f'Full message received from Kafka::\n', message)
    msgAsString = message.value.decode("utf-8")
    if msgAsString == '':
        print('IGNORED SECOND ROW with all commans, CONTINUING to end of the for loop')
        continue
    elif msgAsString[0:8] == 'VendorID':
        print('IGNORED FIRST ROW with columns names, CONTINUING to end of the for loop')
        continue
    elif msgAsString.count(',') == 16:
        msgAsList = msgAsString.split(",")
        trip_distanceAsString = str(msgAsList[4])
        if trip_distanceAsString[0:1] == '.':        # a 0 value comes as  .00  so checking and making as 0.00
            trip_distanceAsString = '0' + trip_distanceAsString
        NYTNewRowJsonBuiltUpAsString='{' +                                      \
        '"VendorID" : ' + str(msgAsList[0]) + ', ' +                            \
        '"tpep_pickup_datetime" : ' + '"' + str(msgAsList[1]) +  '"' + ', ' +   \
        '"tpep_dropoff_datetime" : ' + '"' + str(msgAsList[2]) +  '"' + ', ' +  \
        '"passenger_count" : ' + str(msgAsList[3]) + ', ' +                     \
        '"trip_distance" : ' + trip_distanceAsString + ', ' +                   \
        '"RatecodeID" : ' + str(msgAsList[5]) + ', ' +                          \
        '"store_and_fwd_flag" : ' + '"' + str(msgAsList[6]) + '"' + ', ' +      \
        '"PULocationID" : ' + str(msgAsList[7]) + ', ' +                        \
        '"DOLocationID" : ' + str(msgAsList[8]) + ', ' +                        \
        '"payment_type" : ' + str(msgAsList[9]) + ', ' +                        \
        '"fare_amount" : ' + str(msgAsList[10]) + ', ' +                        \
        '"extra" : ' + str(msgAsList[11]) + ', ' +                              \
        '"mta_tax" : ' + str(msgAsList[12]) + ', ' +                            \
        '"tip_amount" : ' + str(msgAsList[13]) + ', ' +                         \
        '"tolls_amount" : ' + str(msgAsList[14]) + ', ' +                       \
        '"improvement_surcharge" : ' + str(msgAsList[15]) + ', ' +              \
        '"total_amount" : ' + str(msgAsList[16]) + '}'
        #print(f'Msg {msgsReadCount} - JSON built as=\n{NYTNewRowJsonBuiltUpAsString}')
        if ( str(msgAsList[1]) < boundUpperPUDatetime1 and str(msgAsList[1]) >= boundLowerPUDatetime1 ):
            try:
                collection1.insert_one(loads(NYTNewRowJsonBuiltUpAsString))
                #print(f"Collection 1 -- NO ERROR for {msgsReadCount} th message insert")
                countDocsWritten1 = countDocsWritten1 + 1
                if countDocsWritten1 % 50000 == 0:
                    print(f"Collection 1 - write SUCCESS - {msgsReadCount} th message as {countDocsWritten1} th insert")
                if countDocsWritten1 % 250000 == 0:
                    print(f'Msg {msgsReadCount} - JSON built as=\n{NYTNewRowJsonBuiltUpAsString}')
            except Exception as e:
                print(f'Collection 1 ERROR Insert :: {msgsReadCount} th message\nError:: ', type(e), e)
        elif ( str(msgAsList[1]) < boundUpperPUDatetime2 and str(msgAsList[1]) >= boundLowerPUDatetime2 ):
            try:
                collection2.insert_one(loads(NYTNewRowJsonBuiltUpAsString))
                #print(f"Collection 2 -- NO ERROR for {msgsReadCount} th message insert")
                countDocsWritten2 = countDocsWritten2 + 1
                if countDocsWritten2 % 50000 == 0:
                    print(f"Collection 2 - write SUCCESS - {msgsReadCount} th message as {countDocsWritten2} th insert")
                if countDocsWritten2 % 500000 == 0:
                    print(f'Msg {msgsReadCount} - JSON built as=\n{NYTNewRowJsonBuiltUpAsString}')
            except Exception as e:
                print(f'Collection 2 ERROR Insert :: {msgsReadCount} th message\nError:: ', type(e), e)
    else:
        print(f"{msgsReadCount} th message NOT having 16 commas so IGNORED MESSAGE")


print('\nExited FOR LOOP\nRead total %d messages from Kafka' %(msgsReadCount))
print('\nWritten %d documents to Mongo Collection1' %(countDocsWritten1))
print('\nWritten %d documents to Mongo Collection2' %(countDocsWritten2))
print('\nDone processing. Normal exit from program at: ', datetime.now().strftime("%c"))
#
# ways to copy a dictionary value      dict2 = dict(dict1)   OR   dict2 = dict1.copy()
#

In [ ]:
# CONSUMER 2    CONSUMER 2
# -*- coding: utf-8 -*-
"""
Created on Sun Jan  6 11:35:20 2019

@author: RB
"""
from kafka import KafkaConsumer
from pymongo import MongoClient
from json import loads
from datetime import datetime

print('\nStart time of consumer program: ', datetime.now().strftime("%c"))

countDocsWritten1 = 0
countDocsWritten2 = 0
msgsReadCount = 0
topicName = 'TestNYTFullJuneDb10

consumer = KafkaConsumer(
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
#     auto_offset_reset='latest',   # default is latest
     enable_auto_commit=True,
     auto_commit_interval_ms=2000,
     group_id='my-group1')
#     consumer_timeout_ms=5000)   #consumer_timeout_ms (int) – number of milliseconds to block during 
                                 #    message iteration before raising StopIteration (i.e., ending the 
                                 #    iterator). Default block forever [float(‘inf’)].
consumer.subscribe(topicName)

client1 = MongoClient('localhost:27017')
client2 = MongoClient('localhost:27017')
collection1 = client1.TestNYTFullJuneDb11.TestNYTFullJuneColJun01to12
collection2 = client2.TestNYTFullJuneDb11.TestNYTFullJuneColJun13to24

boundLowerPUDatetime1 = '2018-06-01 00:00:00'   # the pickup datetime from csv to be greater or equal to this
boundUpperPUDatetime1 = '2018-06-13 00:00:00'   # the pickup datetime from csv to be lower than this
boundLowerPUDatetime2 = '2018-06-13 00:00:00'   # the pickup datetime from csv to be greater or equal to this
boundUpperPUDatetime2 = '2018-06-25 00:00:00'   # the pickup datetime from csv to be lower than this

for message in consumer:
    msgsReadCount = msgsReadCount + 1
    NYTNewRowJsonBuiltUpAsString = "{}"
#    print(f'Full message received from Kafka::\n', message)
    msgAsString = message.value.decode("utf-8")
    if msgAsString == '':
        print('IGNORED SECOND ROW with all commans, CONTINUING to end of the for loop')
        continue
    elif msgAsString[0:8] == 'VendorID':
        print('IGNORED FIRST ROW with columns names, CONTINUING to end of the for loop')
        continue
    elif msgAsString.count(',') == 16:
        msgAsList = msgAsString.split(",")
        trip_distanceAsString = str(msgAsList[4])
        if trip_distanceAsString[0:1] == '.':        # a 0 value comes as  .00  so checking and making as 0.00
            trip_distanceAsString = '0' + trip_distanceAsString
        NYTNewRowJsonBuiltUpAsString='{' +                                      \
        '"VendorID" : ' + str(msgAsList[0]) + ', ' +                            \
        '"tpep_pickup_datetime" : ' + '"' + str(msgAsList[1]) +  '"' + ', ' +   \
        '"tpep_dropoff_datetime" : ' + '"' + str(msgAsList[2]) +  '"' + ', ' +  \
        '"passenger_count" : ' + str(msgAsList[3]) + ', ' +                     \
        '"trip_distance" : ' + trip_distanceAsString + ', ' +                   \
        '"RatecodeID" : ' + str(msgAsList[5]) + ', ' +                          \
        '"store_and_fwd_flag" : ' + '"' + str(msgAsList[6]) + '"' + ', ' +      \
        '"PULocationID" : ' + str(msgAsList[7]) + ', ' +                        \
        '"DOLocationID" : ' + str(msgAsList[8]) + ', ' +                        \
        '"payment_type" : ' + str(msgAsList[9]) + ', ' +                        \
        '"fare_amount" : ' + str(msgAsList[10]) + ', ' +                        \
        '"extra" : ' + str(msgAsList[11]) + ', ' +                              \
        '"mta_tax" : ' + str(msgAsList[12]) + ', ' +                            \
        '"tip_amount" : ' + str(msgAsList[13]) + ', ' +                         \
        '"tolls_amount" : ' + str(msgAsList[14]) + ', ' +                       \
        '"improvement_surcharge" : ' + str(msgAsList[15]) + ', ' +              \
        '"total_amount" : ' + str(msgAsList[16]) + '}'
        #print(f'Msg {msgsReadCount} - JSON built as=\n{NYTNewRowJsonBuiltUpAsString}')
        if ( str(msgAsList[1]) < boundUpperPUDatetime1 and str(msgAsList[1]) >= boundLowerPUDatetime1 ):
            try:
                collection1.insert_one(loads(NYTNewRowJsonBuiltUpAsString))
                #print(f"Collection 1 -- NO ERROR for {msgsReadCount} th message insert")
                countDocsWritten1 = countDocsWritten1 + 1
                if countDocsWritten1 % 50000 == 0:
                    print(f"Collection 1 - write SUCCESS - {msgsReadCount} th message as {countDocsWritten1} th insert")
                if countDocsWritten1 % 250000 == 0:
                    print(f'Msg {msgsReadCount} - JSON built as=\n{NYTNewRowJsonBuiltUpAsString}')
            except Exception as e:
                print(f'Collection 1 ERROR Insert :: {msgsReadCount} th message\nError:: ', type(e), e)
        elif ( str(msgAsList[1]) < boundUpperPUDatetime2 and str(msgAsList[1]) >= boundLowerPUDatetime2 ):
            try:
                collection2.insert_one(loads(NYTNewRowJsonBuiltUpAsString))
                #print(f"Collection 2 -- NO ERROR for {msgsReadCount} th message insert")
                countDocsWritten2 = countDocsWritten2 + 1
                if countDocsWritten2 % 50000 == 0:
                    print(f"Collection 2 - write SUCCESS - {msgsReadCount} th message as {countDocsWritten2} th insert")
                if countDocsWritten2 % 500000 == 0:
                    print(f'Msg {msgsReadCount} - JSON built as=\n{NYTNewRowJsonBuiltUpAsString}')
            except Exception as e:
                print(f'Collection 2 ERROR Insert :: {msgsReadCount} th message\nError:: ', type(e), e)
    else:
        print(f"{msgsReadCount} th message NOT having 16 commas so IGNORED MESSAGE")


print('\nExited FOR LOOP\nRead total %d messages from Kafka' %(msgsReadCount))
print('\nWritten %d documents to Mongo Collection1' %(countDocsWritten1))
print('\nWritten %d documents to Mongo Collection2' %(countDocsWritten2))
print('\nDone processing. Normal exit from program at: ', datetime.now().strftime("%c"))
#
# ways to copy a dictionary value      dict2 = dict(dict1)   OR   dict2 = dict1.copy()
#

In [ ]:
# CONSUMER 3    CONSUMER 3
# -*- coding: utf-8 -*-
"""
Created on Sun Jan  6 11:35:20 2019

@author: RB
"""
from kafka import KafkaConsumer
from pymongo import MongoClient
from json import loads
from datetime import datetime

print('\nStart time of consumer program: ', datetime.now().strftime("%c"))

countDocsWritten1 = 0
countDocsWritten2 = 0
msgsReadCount = 0
topicName = 'TestNYTFullJuneDb10

consumer = KafkaConsumer(
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
#     auto_offset_reset='latest',   # default is latest
     enable_auto_commit=True,
     auto_commit_interval_ms=2000,
     group_id='my-group1')
#     consumer_timeout_ms=5000)   #consumer_timeout_ms (int) – number of milliseconds to block during 
                                 #    message iteration before raising StopIteration (i.e., ending the 
                                 #    iterator). Default block forever [float(‘inf’)].
consumer.subscribe(topicName)

client1 = MongoClient('localhost:27017')
client2 = MongoClient('localhost:27017')
collection1 = client1.TestNYTFullJuneDb11.TestNYTFullJuneColJun01to12
collection2 = client2.TestNYTFullJuneDb11.TestNYTFullJuneColJun13to24

boundLowerPUDatetime1 = '2018-06-01 00:00:00'   # the pickup datetime from csv to be greater or equal to this
boundUpperPUDatetime1 = '2018-06-13 00:00:00'   # the pickup datetime from csv to be lower than this
boundLowerPUDatetime2 = '2018-06-13 00:00:00'   # the pickup datetime from csv to be greater or equal to this
boundUpperPUDatetime2 = '2018-06-25 00:00:00'   # the pickup datetime from csv to be lower than this

for message in consumer:
    msgsReadCount = msgsReadCount + 1
    NYTNewRowJsonBuiltUpAsString = "{}"
#    print(f'Full message received from Kafka::\n', message)
    msgAsString = message.value.decode("utf-8")
    if msgAsString == '':
        print('IGNORED SECOND ROW with all commans, CONTINUING to end of the for loop')
        continue
    elif msgAsString[0:8] == 'VendorID':
        print('IGNORED FIRST ROW with columns names, CONTINUING to end of the for loop')
        continue
    elif msgAsString.count(',') == 16:
        msgAsList = msgAsString.split(",")
        trip_distanceAsString = str(msgAsList[4])
        if trip_distanceAsString[0:1] == '.':        # a 0 value comes as  .00  so checking and making as 0.00
            trip_distanceAsString = '0' + trip_distanceAsString
        NYTNewRowJsonBuiltUpAsString='{' +                                      \
        '"VendorID" : ' + str(msgAsList[0]) + ', ' +                            \
        '"tpep_pickup_datetime" : ' + '"' + str(msgAsList[1]) +  '"' + ', ' +   \
        '"tpep_dropoff_datetime" : ' + '"' + str(msgAsList[2]) +  '"' + ', ' +  \
        '"passenger_count" : ' + str(msgAsList[3]) + ', ' +                     \
        '"trip_distance" : ' + trip_distanceAsString + ', ' +                   \
        '"RatecodeID" : ' + str(msgAsList[5]) + ', ' +                          \
        '"store_and_fwd_flag" : ' + '"' + str(msgAsList[6]) + '"' + ', ' +      \
        '"PULocationID" : ' + str(msgAsList[7]) + ', ' +                        \
        '"DOLocationID" : ' + str(msgAsList[8]) + ', ' +                        \
        '"payment_type" : ' + str(msgAsList[9]) + ', ' +                        \
        '"fare_amount" : ' + str(msgAsList[10]) + ', ' +                        \
        '"extra" : ' + str(msgAsList[11]) + ', ' +                              \
        '"mta_tax" : ' + str(msgAsList[12]) + ', ' +                            \
        '"tip_amount" : ' + str(msgAsList[13]) + ', ' +                         \
        '"tolls_amount" : ' + str(msgAsList[14]) + ', ' +                       \
        '"improvement_surcharge" : ' + str(msgAsList[15]) + ', ' +              \
        '"total_amount" : ' + str(msgAsList[16]) + '}'
        #print(f'Msg {msgsReadCount} - JSON built as=\n{NYTNewRowJsonBuiltUpAsString}')
        if ( str(msgAsList[1]) < boundUpperPUDatetime1 and str(msgAsList[1]) >= boundLowerPUDatetime1 ):
            try:
                collection1.insert_one(loads(NYTNewRowJsonBuiltUpAsString))
                #print(f"Collection 1 -- NO ERROR for {msgsReadCount} th message insert")
                countDocsWritten1 = countDocsWritten1 + 1
                if countDocsWritten1 % 50000 == 0:
                    print(f"Collection 1 - write SUCCESS - {msgsReadCount} th message as {countDocsWritten1} th insert")
                if countDocsWritten1 % 250000 == 0:
                    print(f'Msg {msgsReadCount} - JSON built as=\n{NYTNewRowJsonBuiltUpAsString}')
            except Exception as e:
                print(f'Collection 1 ERROR Insert :: {msgsReadCount} th message\nError:: ', type(e), e)
        elif ( str(msgAsList[1]) < boundUpperPUDatetime2 and str(msgAsList[1]) >= boundLowerPUDatetime2 ):
            try:
                collection2.insert_one(loads(NYTNewRowJsonBuiltUpAsString))
                #print(f"Collection 2 -- NO ERROR for {msgsReadCount} th message insert")
                countDocsWritten2 = countDocsWritten2 + 1
                if countDocsWritten2 % 50000 == 0:
                    print(f"Collection 2 - write SUCCESS - {msgsReadCount} th message as {countDocsWritten2} th insert")
                if countDocsWritten2 % 500000 == 0:
                    print(f'Msg {msgsReadCount} - JSON built as=\n{NYTNewRowJsonBuiltUpAsString}')
            except Exception as e:
                print(f'Collection 2 ERROR Insert :: {msgsReadCount} th message\nError:: ', type(e), e)
    else:
        print(f"{msgsReadCount} th message NOT having 16 commas so IGNORED MESSAGE")


print('\nExited FOR LOOP\nRead total %d messages from Kafka' %(msgsReadCount))
print('\nWritten %d documents to Mongo Collection1' %(countDocsWritten1))
print('\nWritten %d documents to Mongo Collection2' %(countDocsWritten2))
print('\nDone processing. Normal exit from program at: ', datetime.now().strftime("%c"))
#
# ways to copy a dictionary value      dict2 = dict(dict1)   OR   dict2 = dict1.copy()
#